In [1]:
import random
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from itertools import combinations
from shapely.geometry import Point
from geopy.distance import geodesic
from shapely.wkt import dumps, loads
from shapely.geometry import Polygon
from shapely.wkt import loads as wkt_loads
from scipy.spatial.distance import euclidean

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType, TimestampType
spark = SparkSession.builder.appName("example").config("hive.exec.dynamic.partition", "true").config("hive.exec.dynamic.partition.mode", "nonstrict").enableHiveSupport().getOrCreate()

#### import dataset

In [ ]:
sqldf = spark.sql("SELECT * FROM table A")
parcel_df_all = sqldf.toPandas()
parcel_df = parcel_df_all[parcel_df_all['zone_all'].isna()]
parcel_df

In [14]:
# Sample data for demonstration
data = []

# U.S. latitude and longitude range
min_latitude = 33.0
max_latitude = 41.0
min_longitude = -118.0
max_longitude = -87.0

# Generate 15 rows of fake data
for i in range(15):
    date_id = "2024-02-20"
    sp2_name = random.choice(["STORE1", "STORE2", "STORE3", "STORE4"])  # Example store names
    shipping_time = random.choice(["DAY", "NIGHT"])  # Example shipping times
    backlog_day = random.randint(3, 6)  # Example backlog days
    zone_all = random.choice(["A", "B", "C", "D"])  # Example zone data
    volume = round(random.uniform(2000, 40000), 3)  # Example volume
    shipment_id = f"US{random.randint(100000000000, 999999999999)}"  # Example shipment ID
    delivery_window = random.randint(1, 3)  # Example delivery window
    lat = round(random.uniform(min_latitude, max_latitude), 6)  # Random latitude within U.S. range
    lng = round(random.uniform(min_longitude, max_longitude), 6)  # Random longitude within U.S. range
    
    data.append([date_id, sp2_name, shipping_time, backlog_day, zone_all, volume, shipment_id, delivery_window, lat, lng])

# Define columns for the DataFrame
columns = ['date_id', 'sp2_name', 'shipping_time', 'backlog_day', 'zone_all', 'volume', 'shipment_id', 'delivery_window', 'lat', 'lng']

# Create the DataFrame
parcel_df = pd.DataFrame(data, columns=columns)

# Display the generated DataFrame
print(parcel_df)

       date_id sp2_name shipping_time  backlog_day zone_all     volume  \
0   2024-02-20   STORE3           DAY            6        C  31202.370   
1   2024-02-20   STORE3           DAY            5        C  16449.074   
2   2024-02-20   STORE2           DAY            3        B   5276.623   
3   2024-02-20   STORE3           DAY            4        D  24773.206   
4   2024-02-20   STORE1         NIGHT            3        A   8072.813   
5   2024-02-20   STORE4         NIGHT            3        C  11676.659   
6   2024-02-20   STORE3         NIGHT            5        D  23235.471   
7   2024-02-20   STORE1           DAY            4        B  38794.745   
8   2024-02-20   STORE4           DAY            3        A  13952.333   
9   2024-02-20   STORE1         NIGHT            6        A  31089.251   
10  2024-02-20   STORE1         NIGHT            3        C  16663.823   
11  2024-02-20   STORE3         NIGHT            6        C  16550.581   
12  2024-02-20   STORE4           DAY 

#### Import Store Delivery Restrictions

In [6]:
# Sample data for demonstration
data = [
    ["STORE1", "2023-12-18", "2050-12-31", "DAY", 70, 20, 6, 2, 'D', 2, 1, 2, 1],
    ["STORE2", "2023-12-18", "2023-12-21", "NIGHT", 100, 25, 6, 2, '', 2, 1, 2, 1],
    ["STORE3", "2023-12-18", "2050-12-31", "NIGHT", 95, 25, 6, 2, '', 2, 1, 2, 1],
    ["STORE4", "2023-12-18", "2050-12-31", "NIGHT", 75, 20, 6, 2, '', 2, 1, 2, 1]
]

columns = ['sp_name', 'start_date', 'end_date', 'appsheet_daynight', 'day_parcel', 'night_parcel','day_cabinet', 'night_cabinet', 'priority_zone', 'weekday_penalty_cabinet_day','weekday_penalty_cabinet_night', 'weekend_penalty_cabinet_day','weekend_penalty_cabinet_night']
limit = pd.DataFrame(data, columns=columns)
limit['start_date'] = pd.to_datetime(limit['start_date'])
limit['end_date'] = pd.to_datetime(limit['end_date'])
limit

,sp_name,start_date,end_date,appsheet_daynight,day_parcel,night_parcel,day_cabinet,night_cabinet,priority_zone,weekday_penalty_cabinet_day,weekday_penalty_cabinet_night,weekend_penalty_cabinet_day,weekend_penalty_cabinet_night
0,STORE1,2023-12-18,2050-12-31,DAY,70,20,6,2,D,2,1,2,1
1,STORE2,2023-12-18,2023-12-21,NIGHT,100,25,6,2,,2,1,2,1
2,STORE3,2023-12-18,2050-12-31,NIGHT,95,25,6,2,,2,1,2,1
3,STORE4,2023-12-18,2050-12-31,NIGHT,75,20,6,2,,2,1,2,1


#### Find nearby zone

In [ ]:
areas = zone_df
areas.iloc[1]

In [ ]:
def load_wkt(geometry):
    # Check if the input is already a WKT string, and return it
    if isinstance(geometry, str):
        return geometry
    # If it's a Polygon object, convert it to WKT and return
    elif geometry.geom_type == 'Polygon':
        return dumps(geometry)
    else:
        # Handle other geometry types if needed
        return None  # or raise an exception

In [ ]:
# Apply the load_wkt function to the 'geometry' column
areas['geometry'] = areas['geometry'].apply(load_wkt)

# Create a list of tuples with ('sp2_name', 'zone_name') and 'geometry'
combined_list = [(row['sp2_name'], row['zone_name'], row['geometry']) for _, row in areas.iterrows()]

# Create a dictionary with tuples as keys and entire row as values
# areas_dict_polygons = {(sp2_name, zone_name): row for sp2_name, zone_name, row in combined_list}
areas_dict_polygons = {(sp2_name, zone_name): {'geometry': loads(geometry_wkt), 'zone_order': None} 
                       for sp2_name, zone_name, geometry_wkt in combined_list}
areas_dict_polygons

In [9]:
# Sample data for demonstration
min_lat, max_lat = 33.0, 41.0
min_lng, max_lng = -118.0, -87.0

# Function to generate random polygons
def generate_polygon():
    num_points = random.randint(3, 6)  # Random number of points for the polygon
    points = [(random.uniform(min_lng, max_lng), random.uniform(min_lat, max_lat)) for _ in range(num_points)]
    return Polygon(points)

# Generate fake data
data = {}
stores = ['STORE1', 'STORE2', 'STORE3', 'STORE4']
letters = ['A', 'B', 'C', 'D']

for store in stores:
    for letter in letters:
        key = (store, f"{letter}-{random.randint(1, 12)}")
        data[key] = {
            'geometry': generate_polygon(),
            'zone_order': None  # Placeholder for zone order data
        }

# Convert dictionary to DataFrame
areas_dict_polygons = pd.DataFrame.from_dict(data, orient='index')
areas_dict_polygons.reset_index(inplace=True)
areas_dict_polygons.columns = ['sp_name', 'zone_name', 'geometry', 'zone_order']

print(areas_dict_polygons.head())  # Print the first few rows of the DataFrame

  sp_name zone_name                                           geometry  \
0  STORE1       A-3  POLYGON ((-107.35662282555718 38.4000637759015...   
1  STORE1       B-9  POLYGON ((-105.74959140699993 35.0862345638136...   
2  STORE1       C-8  POLYGON ((-91.52447343151839 39.29440957979595...   
3  STORE1       D-8  POLYGON ((-97.12077360811219 34.11370340257398...   
4  STORE2       A-6  POLYGON ((-112.84888878882268 38.6079094285059...   

  zone_order  
0       None  
1       None  
2       None  
3       None  
4       None  


In [10]:
box_volume = 1600
sp2_list = parcel_df['sp2_name'].unique()
sp2_list

array(['STORE3', 'STORE1', 'STORE2'], dtype=object)

In [11]:
shipment = parcel_df
shipment = shipment.drop_duplicates()
shipment['backlog_day'] = shipment['backlog_day'].astype(int)
shipment['volume'] = shipment['volume'].astype(float)

# Find rows where 'zone_all' and 'backlog_day' are not null, and create a new DataFrame for filling missing values
mask = (~shipment['zone_all'].isnull()) & (~shipment['backlog_day'].isnull())
non_null_values = shipment[mask]

# Find rows where 'backlog_day' is greater than or equal to 4 and 'zone_all' is null, and retrieve 'backlog_day' and 'zone_all' values
additional_values_to_fill = shipment[(shipment['backlog_day'] >= 4) & (shipment['zone_all'].isnull())][['backlog_day', 'zone_all']]

# Store values to be filled as a dictionary, using 'backlog_day' as the key and 'zone_all' as the value
additional_values_to_fill_dict = additional_values_to_fill.set_index('backlog_day')['zone_all'].to_dict()

# Use mask to apply the filling conditions
mask_to_fill = (shipment['backlog_day'] >= 4) & shipment['zone_all'].isnull()
shipment.loc[mask_to_fill, 'zone_all'] = shipment.loc[mask_to_fill, 'backlog_day'].map(additional_values_to_fill_dict)

# Check for remaining NaN values
na_values_after_fill = shipment.loc[mask_to_fill, 'zone_all'].isnull().sum()

# If there are still NaN values, fill them with 'A'
if na_values_after_fill > 0:
    shipment.loc[mask_to_fill & shipment['zone_all'].isnull(), 'zone_all'] = 'A'

# Filter rows where 'zone_all' is still null (no zone assigned)
no_zone = shipment[shipment['zone_all'].isnull()]
no_zone = no_zone[['shipment_id', 'backlog_day', 'volume', 'shipping_time', 'zone_all']]

# Drop rows with NaN values in 'zone_all'
shipment.dropna(subset=['zone_all'], inplace=True)

#### Main package placement algorithm

In [12]:
def pack_boxes_with_order(sorted_groups, box_volume, zone_orders):
    placed_shipments = set()  # Set to track placed shipments
    all_box = []  # List to store all boxes
    box_count = 0  # Counter for box IDs

    while sorted_groups:
        for zone in sorted_groups:
            current_box_volume = 0  # Current volume of the box being packed
            box_count += 1  # Increment box count for each new box
            current_box = []  # List to store shipments in the current box
            neighbor_cannot_put = 0  # Counter for neighbors that cannot contribute parcels

            if zone in sorted_groups:
                # Iterate through parcels in the current zone
                for index, row in sorted_groups[zone].iterrows():
                    # Check conditions to place a shipment in the current box
                    if (row['shipment_id'] not in placed_shipments and 
                        current_box_volume + row['volume'] <= box_volume and 
                        unique_shipments_per_zone.loc[zone, 'parcel_cnt'] > 0):
                        
                        # Add shipment details to current box
                        current_box.append({
                            'box_id': f'{zone}{box_count}',
                            'volume': row['volume'],
                            'shipment_id': row['shipment_id'],
                            'backlog_day': row['backlog_day'],
                            'shipping_time': row['shipping_time'],
                            'zone_all': row['zone_all'],
                            'lat': row['lat'],
                            'lng': row['lng'],
                            'sp2_name': row['sp2_name'],
                            'remaining_volume': box_volume - (current_box_volume + row['volume'])
                        })
                        current_box_volume += row['volume']  # Update current box volume
                        placed_shipments.add(row['shipment_id'])  # Mark shipment as placed
                        unique_shipments_per_zone.loc[zone, 'parcel_cnt'] -= 1  # Decrease parcel count

                # Check neighboring zones for parcels
                for neighbor in zone_orders[zone].split(','):
                    neighbor_parcels = sorted_groups.get(neighbor)
                    if neighbor_parcels is None or neighbor_parcels.empty:
                        continue  # Skip if neighbor zone has no parcels

                    neighbor_total_volume = sum(neighbor_parcels['volume'])
                    # Try to add parcels from neighboring zone to current box
                    if current_box_volume + neighbor_total_volume <= box_volume:
                        for index, parcel in neighbor_parcels.iterrows():
                            if (current_box_volume + parcel['volume'] <= box_volume and 
                                parcel['shipment_id'] not in placed_shipments):
                                
                                # Add neighboring parcel details to current box
                                current_box.append({
                                    'box_id': f'{zone}{box_count}',
                                    'volume': parcel['volume'],
                                    'shipment_id': parcel['shipment_id'],
                                    'backlog_day': parcel['backlog_day'],
                                    'shipping_time': parcel['shipping_time'],
                                    'zone_all': parcel['zone_all'],
                                    'remaining_volume': box_volume - (current_box_volume + parcel['volume']),
                                    'lat': parcel['lat'],
                                    'lng': parcel['lng'],
                                    'sp2_name': parcel['sp2_name']
                                })
                                current_box_volume += parcel['volume']  # Update current box volume
                                placed_shipments.add(parcel['shipment_id'])  # Mark shipment as placed
                                unique_shipments_per_zone.loc[neighbor, 'parcel_cnt'] -= 1  # Decrease parcel count

                    # Count neighboring zones that cannot contribute parcels
                    if unique_shipments_per_zone.loc[neighbor, 'parcel_cnt'] > 0:
                        neighbor_cannot_put += 1
                    if neighbor_cannot_put >= 3:
                        break  # Break if more than 3 neighbors cannot contribute

                # Append current box to all boxes
                all_box.append(current_box)

                # Update remaining shipments and re-group by 'zone_all'
                remaining_shipments = shipment_day[~shipment_day['shipment_id'].isin(placed_shipments)]
                grouped = remaining_shipments.groupby('zone_all')
                sorted_groups = {
                    zone: group.sort_values(by=['backlog_day', 'shipping_time', 'volume'], ascending=[False, False, False])
                    for zone, group in grouped
                }

                # Sort zones by maximum backlog day
                max_backlog_per_zone = {zone: group['backlog_day'].max() for zone, group in grouped}
                sorted_zones = sorted(max_backlog_per_zone.keys(), key=lambda x: max_backlog_per_zone[x], reverse=True)
                sorted_groups = {zone: sorted_groups[zone] for zone in sorted_zones if zone in sorted_groups}
                break  # Exit loop after processing one zone

    # Create DataFrames for each box and filter out empty DataFrames
    dfs = [pd.DataFrame(sublist, columns=['box_id', 'volume', 'shipment_id', 'backlog_day', 'shipping_time', 'zone_all', 'lat', 'lng', 'sp2_name', 'remaining_volume']) for sublist in all_box]
    dfs = [df for df in dfs if not df.empty]

    result = pd.DataFrame()  # Initialize empty DataFrame for concatenation
    if dfs:  # Concatenate DataFrames if list is not empty
        result = pd.concat(dfs, ignore_index=True)

    # Update remaining shipments after packing
    remaining_shipments = shipment_day[~shipment_day['shipment_id'].isin(placed_shipments)]
    
    return result, placed_shipments, remaining_shipments

#### Result

In [19]:
sp_done = []  # List to store processed SPs
temp_df = pd.DataFrame()  # Temporary DataFrame (not used in current snippet)
full_df = pd.DataFrame()  # Full DataFrame (not used in current snippet)
result_df = pd.DataFrame()  # Result DataFrame to store final selected data
current_date = datetime.datetime.now().date()  # Get current date

for sp in sp2_list:
    print('sp', sp)
    try:
    #-------------------------------------------------------------------------------------------------------------------------------    
        # DAY shipments
        # Filter shipments for the current SP excluding NIGHT shipments
        shipment_day = shipment[(shipment['shipping_time']!='NIGHT') & (shipment['sp2_name'] == sp)]
        # Group shipments by 'zone_all' and sort within each group
        grouped = shipment_day.groupby('zone_all')
        sorted_groups = {
            zone: group.sort_values(by=['zone_all', 'backlog_day', 'shipping_time', 'volume'], ascending=[True, False, False, False])
            for zone, group in grouped
        }    
        # Determine maximum backlog per zone
        max_backlog_per_zone = {zone: group['backlog_day'].max() for zone, group in grouped}
        # Fetch priority zone for the SP and prioritize it in sorting
        priority_zone = limit[(limit['sp_name'] == sp)]['priority_zone'].values[0]
        if priority_zone in max_backlog_per_zone:
            sorted_zones = sorted(max_backlog_per_zone.keys(), key=lambda x: (x != priority_zone, max_backlog_per_zone[x]), reverse=True)
            sorted_zones.remove(priority_zone)
            sorted_zones.insert(0, priority_zone)
        else:
            sorted_zones = sorted(max_backlog_per_zone.keys(), key=lambda x: max_backlog_per_zone[x], reverse=True)
        # Filter and sort groups based on prioritized zones
        sorted_groups = {zone: sorted_groups[zone] for zone in sorted_zones if zone in sorted_groups}
        # Count unique shipments per zone
        unique_shipments_per_zone = shipment_day.groupby('zone_all')['shipment_id'].nunique()
        unique_shipments_per_zone = pd.DataFrame({
            'parcel_cnt': unique_shipments_per_zone.values
        }, index=unique_shipments_per_zone.index)
        # Get zone orders from areas_dict_polygons for the current SP
        zone_orders = {}
        for (sp2_name, area), data in areas_dict_polygons.items():
            if sp2_name == sp:
                zone_order = data.get('zone_order', None)
                if zone_order is not None:
                    zone_orders[area] = zone_order    
        # Call the packing function to allocate shipments to boxes
        result, placed_shipments, remaining_shipments = pack_boxes_with_order(sorted_groups, box_volume, zone_orders)
        sp_done.append(sp)  # Add SP to processed list
    #-------------------------------------------------------------------------------------------------------------------------------
    # OMITTED 
    #-------------------------------------------------------------------------------------------------------------------------------
        # Fetch parcel limits for day and night based on current date
        night_parcel_values = limit[
            (limit['sp_name'] == sp) &
            (limit['start_date'].dt.date <= current_date) &
            (limit['end_date'].dt.date >= current_date)
        ]['night_parcel'].values
        if len(night_parcel_values) == 0:
            night_parcel_limit = int(0)
        if len(night_parcel_values) > 0:
            night_parcel_limit = int(night_parcel_values[0])

        day_parcel_values = limit[
            (limit['sp_name'] == sp) &
            (limit['start_date'].dt.date <= current_date) &
            (limit['end_date'].dt.date >= current_date)
        ]['day_parcel'].values
        if len(day_parcel_values) == 0:
            day_parcel_limit = int(0)
        if len(day_parcel_values) > 0:
            day_parcel_limit = int(day_parcel_values[0])

        day_remain_parcel = day_parcel_limit
        night_remain_parcel = night_parcel_limit

        # Separate day and night shipments from result based on box_id
        day_full_df = result.loc[result['box_id'].astype(str).str.isnumeric()]
        night_full_df = result.loc[~result['box_id'].astype(str).str.isnumeric()]

        day_selected = pd.DataFrame()  # DataFrame to store selected day shipments
        night_selected = pd.DataFrame()  # DataFrame to store selected night shipments

        # Count occurrences of each box_id in day_full_df
        day_counts = day_full_df['box_id'].value_counts().sort_index()
        if not night_full_df.empty:
            # Count occurrences of each box_id in night_full_df
            night_counts = night_full_df['box_id'].value_counts().sort_index()
            # Select night shipments based on remaining night parcel limit
            for box_id, count in night_counts.items():
                if night_remain_parcel >= count or count > night_remain_parcel > 0:
                    night_selected = pd.concat([night_selected, night_full_df[night_full_df['box_id']==box_id]])
                    night_remain_parcel -= count
                elif night_remain_parcel <= 0:
                    break
        # Select day shipments based on remaining day parcel limit
        for box_id, count in day_counts.items():
            if day_remain_parcel >= count or count > day_remain_parcel > 0:
                day_selected = pd.concat([day_selected, day_full_df[day_full_df['box_id']==box_id]])
                day_remain_parcel -= count
            elif day_remain_parcel <= 0:
                break

        selected = pd.concat([day_selected, night_selected])  # Combine selected day and night shipments
    #-------------------------------------------------------------------------------------------------------------------------------   
        # Fetch trip limits for day and night based on current date
        night_trip_values = limit[
            (limit['sp_name'] == sp) &
            (limit['start_date'].dt.date <= current_date) &
            (limit['end_date'].dt.date >= current_date)
        ]['night_cabinet'].values
        if len(night_trip_values) > 0:
            night_trip_limit = int(0)
        if len(night_trip_values) > 0:
            night_trip_limit = int(night_trip_values[0])

        day_parcel_values = limit[
            (limit['sp_name'] == sp) &
            (limit['start_date'].dt.date <= current_date) &
            (limit['end_date'].dt.date >= current_date)
        ]['day_cabinet'].values
        if len(day_parcel_values) > 0:
            day_trip_limit = int(0)
        if len(day_parcel_values) > 0:
            day_trip_limit = int(day_parcel_values[0])

        day_df = selected.loc[selected['box_id'].astype(str).str.isnumeric()]
        night_df = selected.loc[~selected['box_id'].astype(str).str.isnumeric()]
        night_count = selected.loc[~selected['box_id'].astype(str).str.isnumeric()]['box_id'].nunique()
        selected_data = []
        placed = set()
        # Select day shipments based on remaining day trip limit
        for index, row in day_df.iterrows():
            for i in range(day_trip_limit + 1):
                if row['box_id'] <= i and row['shipment_id'] not in placed:
                    selected_data.append(row)
                    placed.add(row['shipment_id'])

        # Select night shipments based on remaining night trip limit
        for index, row in night_df.iterrows():
            for i in range(night_trip_limit + 1):
                number = int(row['box_id'].split('_')[1])
                if number <= i and row['shipment_id'] not in placed:
                    selected_data.append(row)
                    placed.add(row['shipment_id'])
        
        result_df = pd.DataFrame(result)  # Store result DataFrame for the current SP
    except Exception as e:
        print("Error occurred:", e)
        continue  # Continue to the next SP if an error occurs

print(result_df)  # Print the final result DataFrame

Data for sp2_name: STORE4
    box_id     shipment_id  backlog_day     volume shipping_time zone_all  \
0        1  US249242110835            2  16846.795         NIGHT      B-3   
11      12  US248845306474            4  28824.693         NIGHT      A-3   
13      14  US241392924215            5  23688.427           DAY      C-9   

          lat         lng sp2_name origin_box_id  
0   37.755166 -101.165646   STORE4           B-1  
11  40.688958  -89.722076   STORE4          B-12  
13  35.865635 -114.791731   STORE4          B-14  


Data for sp2_name: STORE2
    box_id     shipment_id  backlog_day     volume shipping_time zone_all  \
1        2  US249605809915            4  26003.967           DAY      C-6   
2        3  US245706605854            5   3963.633         NIGHT      C-1   
3        4  US242071389707            5  13349.571           DAY     B-10   
4        5  US245057029796            2  14205.345         NIGHT      D-4   
6        7  US242918249269            2   1644.1